In [1]:
import gc
import math
import resource
import sqlite3
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa
from pyarrow.parquet import ParquetFile
from tqdm import tqdm

In [2]:
# メモリ上限を 6 GB に設定
resource.setrlimit(resource.RLIMIT_DATA, (6 * 2**30, -1))
resource.getrlimit(resource.RLIMIT_DATA)

(6442450944, -1)

In [3]:
# df = pd.read_csv("../data/large_table.csv")
# ↓そのまま実行した結果
# 現在のセルまたは前のセルでコードを実行中に、カーネルがクラッシュしました。


In [4]:
for i, df_i in enumerate(pd.read_csv("../data/large_table.csv", chunksize=100_000)):
    print(df_i.index[-1])
    if i >= 2:
        break


99999
199999
299999


In [5]:
df_i.info()
# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 100000 entries, 200000 to 299999
# Data columns (total 10 columns):
#  #   Column       Non-Null Count   Dtype
# ---  ------       --------------   -----
#  0   id           100000 non-null  int64
#  1   status       74950 non-null   object
#  2   temperature  100000 non-null  float64
#  3   humidity     100000 non-null  float64
#  4   ax           100000 non-null  float64
#  5   ay           100000 non-null  float64
#  6   az           100000 non-null  float64
#  7   gx           100000 non-null  float64
#  8   gy           100000 non-null  float64
#  9   gz           100000 non-null  float64
# dtypes: float64(8), int64(1), object(1)
# memory usage: 7.6+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 200000 to 299999
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           100000 non-null  int64  
 1   status       74950 non-null   object 
 2   temperature  100000 non-null  float64
 3   humidity     100000 non-null  float64
 4   ax           100000 non-null  float64
 5   ay           100000 non-null  float64
 6   az           100000 non-null  float64
 7   gx           100000 non-null  float64
 8   gy           100000 non-null  float64
 9   gz           100000 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 7.6+ MB


---


In [9]:
parquet_file = ParquetFile(source="../data/large_table.parquet")
batches = parquet_file.iter_batches(batch_size=100_000)
for i in range(3):
    rows = next(batches, None)
    df_i: pd.DataFrame = pa.Table.from_batches([rows]).to_pandas()
    print(df_i.shape)
    if i >= 2:
        break


(100000, 10)
(100000, 10)
(100000, 10)


In [10]:
df_i


,id,status,temperature,humidity,ax,ay,az,gx,gy,gz
time_stamp,,,,,,,,,,
2022-01-03 07:33:20,200000,c,20.185232,51.802753,-1.138367e-13,4.466522e-01,7.495994e-01,-3.778582e-14,0.088047,0.082253
2022-01-03 07:33:21,200001,b,20.222187,50.321580,3.151052e-02,1.800760e+00,1.689817e+00,8.096675e-03,0.086312,0.083666
2022-01-03 07:33:22,200002,c,20.736086,50.019283,3.026316e-01,1.532837e+00,5.000509e-01,1.239800e-02,0.160697,0.021002
2022-01-03 07:33:23,200003,c,20.858980,49.884981,2.905752e-01,1.126353e+00,5.320807e-01,1.705069e-02,0.123702,0.048585
2022-01-03 07:33:24,200004,c,20.695456,49.421330,2.220936e-01,2.417131e-02,4.005936e-01,9.960867e-04,0.098473,0.096380
...,...,...,...,...,...,...,...,...,...,...
2022-01-04 11:19:55,299995,a,19.988842,50.983008,-3.677839e-01,8.395948e-13,9.369552e-13,-2.245331e-02,0.158117,0.003337
2022-01-04 11:19:56,299996,b,19.987926,51.758059,-4.615545e-01,2.457543e-01,7.202788e-01,-2.131632e-02,0.107261,0.038821
2022-01-04 11:19:57,299997,c,20.017151,51.064296,-7.819776e-01,1.371421e-01,1.592785e+00,-2.020288e-02,0.102799,0.035106


In [11]:
df_i.info()
# <class 'pandas.core.frame.DataFrame'>
# DatetimeIndex: 100000 entries, 2022-01-03 07:33:20 to 2022-01-04 11:19:59
# Data columns (total 10 columns):
#  #   Column       Non-Null Count   Dtype
# ---  ------       --------------   -----
#  0   id           100000 non-null  int64
#  1   status       74950 non-null   object
#  2   temperature  100000 non-null  float64
#  3   humidity     100000 non-null  float64
#  4   ax           100000 non-null  float64
#  5   ay           100000 non-null  float64
#  6   az           100000 non-null  float64
#  7   gx           100000 non-null  float64
#  8   gy           100000 non-null  float64
#  9   gz           100000 non-null  float64
# dtypes: float64(8), int64(1), object(1)
# memory usage: 8.4+ MB

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100000 entries, 2022-01-03 07:33:20 to 2022-01-04 11:19:59
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           100000 non-null  int64  
 1   status       74950 non-null   object 
 2   temperature  100000 non-null  float64
 3   humidity     100000 non-null  float64
 4   ax           100000 non-null  float64
 5   ay           100000 non-null  float64
 6   az           100000 non-null  float64
 7   gx           100000 non-null  float64
 8   gy           100000 non-null  float64
 9   gz           100000 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 8.4+ MB


---


In [13]:
chunksize = 1_000_000
df_iter = pd.read_csv("../data/large_table.csv", chunksize=chunksize)
num_records = 0

while True:
    df_i = next(df_iter, None)
    if df_i is None:
        break
    num_records += df_i.__len__()
    del df_i
    gc.collect()

In [14]:
num_records
# 62985601

62985601

In [15]:
num_iter = math.ceil(num_records / chunksize)
num_iter
# 63

63

In [16]:
database_path = Path("../data/large_table.sqlite3")
database_path.unlink(missing_ok=True)

con = sqlite3.connect(database=database_path)
table_name = "test"
# tmp.to_sql(name=table_name, con=con, if_exists="append")

df_iter = pd.read_csv("../data/large_table.csv", chunksize=chunksize)

for _ in tqdm(range(num_iter)):
    df_i = next(df_iter, None)
    if df_i is None:
        break
    df_i.to_sql(name=table_name, con=con, if_exists="append")
    del df_i
    gc.collect()

con.close()
# 100%|██████████| 63/63 [08:41<00:00,  8.27s/it]

100%|██████████| 63/63 [08:41<00:00,  8.27s/it]


In [17]:
try:
    del df_i, df_iter
except Exception:
    pass
gc.collect()

0

In [18]:
con = sqlite3.connect(database=database_path)
df_iter = pd.read_sql(sql="SELECT * FROM test;", con=con, chunksize=chunksize)
num_records_2 = 0
while True:
    df_i = next(df_iter, None)
    if df_i is None:
        break
    num_records_2 += df_i.__len__()
    del df_i
    gc.collect()
num_records_2
# 62985601

62985601

In [19]:
num_records == num_records_2
# True

True